In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from gensim.models import Word2Vec
from sklearn.manifold import TSNE 


Using TensorFlow backend.


In [37]:
#Load in skills dataframe
skill_df = pd.read_csv('skill.tsv', sep='\t').drop('Unnamed: 0', axis=1)
#Load in dictionary associating skill numbers with skill names
skill_dict = {}
with open('skill_dict.json', 'r', encoding='utf-8') as f:
    loaded = json.load(f)
    for v, k in loaded.items():
        skill_dict[k] = str(v) #Use number as key, string as value

In [11]:
#Read out the "sentences"
sentences=skill_df.iloc[:,1:].values.astype(str)
sentences=sentences.tolist()

In [4]:
#Each student is a "sentence", each skill is a "word"
#size = dimensionality of feature vectors
#window = max distance between current and predicted word within a sentence
#min_count = minimum number of occurrences within dataset
#workers = number of threads used
#sg = 0 (CBOW, default); = 1 (skip-gram)
model = Word2Vec(sentences, size=100, window=5, min_count=10, workers=4, sg=1, iter=30)

In [50]:
skill_num=model.wv.vocab; #Names of the words (numbers)
skill_vec=model[skill_num] #Access the vectors

skill_name=list()
#Associate with readable words
for k,v in skill_num.items(): #Iterate over the vocab from word2vec (k = key = number string)
    skill_name.append(skill_dict.get(k)) #Get the value (tag) saved at that key in the other dict
#print(skill_name)

In [7]:
tsne=TSNE() #Instantiate the TSNE model (can change params here)
skill_tsne=tsne.fit_transform(skill_vec) #Run tsne

In [62]:
#Save as a tsv file for d3-scatterplot
# d={'x': skill_tsne[:,0],
#   'y': skill_tsne[:,1],
#   'skill' : skill_name}
tsne_save=pd.DataFrame({'x': skill_tsne[:,0],
  'y': skill_tsne[:,1],
  'skill' : skill_name})
tsne_save.to_csv('tsne_skills.tsv',sep='\t',index=False,columns=['x','y','skill'])

In [61]:
help(tsne_save.to_csv)

Help on method to_csv in module pandas.core.frame:

to_csv(path_or_buf=None, sep=',', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, mode='w', encoding=None, compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, date_format=None, doublequote=True, escapechar=None, decimal='.') method of pandas.core.frame.DataFrame instance
    Write DataFrame to a comma-separated values (csv) file
    
    Parameters
    ----------
    path_or_buf : string or file handle, default None
        File path or object, if None is provided the result is returned as
        a string.
    sep : character, default ','
        Field delimiter for the output file.
    na_rep : string, default ''
        Missing data representation
    float_format : string, default None
        Format string for floating point numbers
    columns : sequence, optional
        Columns to write
    header : boolean or list of string, default T